<a href="https://colab.research.google.com/github/Muneeb-Alvi/Stock-Scanner/blob/main/Stock_Scanner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import time
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

In [ ]:
aletheia_key = "2a621Cb5379544ad93d793b04097b34e"
marketaux_key = "dCBnElYzvxtz9Jzk2NWDqKKPd4KhEzJSFpyNSyUO"

In [71]:
def create_session():
    session = requests.Session() #initialize a new session
    retries = Retry(total=3, backoff_factor=0.1, status_forcelist=[429, 500, 502, 503, 504]) #retry rules: 3 times only after first failure, exponential backoff for retries with factor 0f 0.1
    #common urls mentioned which are caused by rate limit being exceeeded
    session.mount('https://', HTTPAdapter(max_retries=retries)) #all https requests will follow the above retry rules
    return session

In [72]:
def get_stock_data(symbol, session): #get stock data using aletheia api
  url = f"https://api.aletheiaapi.com/StockData?symbol={symbol}"
  headers = {
    "key": aletheia_key,
    "Accept-Version": "2"
  }
  try:
      response = session.get(url, headers=headers, timeout=10) #send request to url using session
      response.raise_for_status() #check status code of response
      return response.json()
  except requests.exceptions.RequestException as e:
      print(f"Error retrieving stock data: {e}")
      return None

In [73]:
def get_stock_news(symbol, session): #get stock news from marketaux api
  url = f"https://api.marketaux.com/v1/news/all?symbols={symbol}&summary=true&api_token={marketaux_key}&language=en"
  response = requests.get(url)
  try:
      response = session.get(url, timeout=10)
      response.raise_for_status()
      return response.json()["data"]
  except requests.exceptions.RequestException as e:
      print(f"Error retrieving news data: {e}")
      return None
  except KeyError:
      print("Unexpected response structure from news API")
      return None


In [74]:
def display_stock_data(symbol):
  session = create_session()
  stock_data = get_stock_data(symbol, session)
  news_sentiment = get_stock_news(symbol, session)
  total_sentiment = 0

  if not stock_data or not news_sentiment: #if one piece of data missing then send error message
        print("Failed to retrieve complete data for the symbol.")
        return

  print(f"Stock Symbol: {symbol}:") #print basic data of the stock
  print(f"Ask Price: {stock_data.get('AskPrice', 'N/A')}")
  print(f"Bid Price: {stock_data.get('BidPrice', 'N/A')}")
  print(f"Percentage Change: {stock_data.get('ChangePercent', 'N/A')}%")

  print("Relevant News & Sentiment:") #print the news title, sentiment and description
  total_sentiment = 0
  relevant_news_count = 0

  for news in news_sentiment:
      for entity in news.get("entities", []):
          if entity["symbol"] == symbol.upper():
              sentiment = entity.get("sentiment_score")
              if sentiment is not None:
                  total_sentiment += sentiment
                  relevant_news_count += 1
                  print(f"- {news['title']} | Sentiment: {sentiment}")
                  print(f"  Description: {news.get('description', 'N/A')}")
  if relevant_news_count > 0: #prevent divide by 0 error if no news found
      avg_sentiment = format(total_sentiment / relevant_news_count, ".4f")
      print(f"Average Sentiment: {avg_sentiment}")
  else:
      print("No relevant news found for sentiment analysis.")

In [78]:
symbol = "" #example with $NVDA, $TSLA, $AAPL, $AMZN
while (symbol != "quit"):
  symbol = input("Enter a stock symbol (or 'quit' to exit): ")
  if symbol != "quit":
    display_stock_data(symbol)
  time.sleep(1)


Enter a stock symbol (or 'quit' to exit): nvda
Stock Symbol: nvda:
Ask Price: 113.26
Bid Price: 103.0
Percentage Change: 0.0153%
Relevant News & Sentiment:
- QQQ: My In-Depth Look At The Nasdaq-100 Fundamentals (NASDAQ:QQQ) | Sentiment: 0.29295
  Description: Invesco QQQ Trust's valuation is at least partially justified, evidenced by its stronger growth, quality, momentum, and sentiment features. Learn more on QQQ ETF here.
- Dow Jones Futures: Trump-Harris Debate Due With S&P 500 Near Key Level; Tesla Jumps | Sentiment: 0.072133
  Description: Dow Jones futures: JPMorgan led banks lower while Tesla jumped in mixed stock market action. The Trump-Harris debate and CPI inflation loom.
- JFrog CEO Shlomi Ben Haim sells shares worth over $470k By Investing.com | Sentiment: 0.6486
  Description: JFrog CEO Shlomi Ben Haim sells shares worth over $470k
Average Sentiment: 0.3379
Enter a stock symbol (or 'quit' to exit): tsla
Stock Symbol: tsla:
Ask Price: 227.0
Bid Price: 213.59
Percentage Cha